In [1]:
import random
import numpy as np
import math
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
class Candidate(object):
    """The candidate class, containing the candidate name, pol_x and pol_y dimensions"""
    
    def __init__(self, pol_x: float, pol_y: float, name, reelection):
        '''
        Initialize Candidate with some base class attributes and a method
        '''
        
        self._candidate_id = name # candidate id
        self.pol_x = pol_x
        self.pol_y = pol_y
        self.run_reelection = reelection
        
    def __repr__(self):
        return 'Candidate({0}, {1}, {2}, {3})'.format(self._candidate_id, self.pol_x, self.pol_y, self.run_reelection)
    

In [3]:
class BaseVoter(object):
    '''
    BaseVoter is the Abstract Base Class for the Greedy, Primary Optimizing and Center Informed Voter Classes.
    
    '''

    def __init__(self, pol_x: float, pol_y: float, name):
        '''
        Initialize BaseVoter with some base class attributes and a method
        '''
        self._voter_id = name # voter name or numerical id
        self.candidate_ranking = dict() # calculated candidate rankings
        self.candidate_choice = None # calculated candidate choice
        self.pol_x = pol_x
        self.pol_y = pol_y
        self._sequence = 0
        self._score = 0
        self.voter_type = 'BaseVoter'

    def __repr__(self):
        return 'Voter({0}, {1}, {2}, {3}, {4})'.format(self.voter_type, self._voter_id, self.pol_x, self.pol_y, self._score)
    
    def calculate_distance(self, candidate : Candidate):
        distance = ( (candidate.pol_x - self.pol_x)**2 + (candidate.pol_y - self.pol_y)**2 )**(0.5)
        return distance
    
    def _calculate_distance(self, x: float, y: float, pol_x: float, pol_y, float):
        distance = ( (x - pol_x)**2 + (y - pol_y)**2 )**(0.5)
        return distance

    def current_score(self):
        return self._score

    def calculate_reward(self, candidate):
        """ calculate, but do not award, the reward function """
        dist = self.calculate_distance(candidate)
        rwd = 0.5 - dist**2
        
        return rwd
    
    def reward(self, candidate):
        """ reward function for successful candidate """
        rwd = self.calculate_reward(candidate)
        self._score += rwd
        return rwd
    
    def closest_candidate(self, candidatelist):
        """ get the closest candidate from the list """
        scores = [self.calculate_distance(candidate) for candidate in candidatelist]
        index_max = np.argmax(scores)
        closest = candidatelist[index_max]
        return closest
        
        # graph and explain reward function choice

In [4]:
class GreedyVoter(BaseVoter):
    '''
    Greedy Voter always chooses the option closest to the voter in every primary election and general election.
    '''
    def __init__(self, pol_x: float, pol_y: float, name):
        
        '''
        Initialize GreedyVoter
        '''
        
        BaseVoter.__init__(self, pol_x, pol_y, name)
        self.voter_type = 'GreedyVoter'

        
    def vote_choice(self, candidatelist, voterchoice=None):
        self.candidate_choice = None # reset
        
#         print(scores)
        
        i = self.closest_candidate(candidatelist)
        self.candidate_choice = i
        
        return i
        

In [5]:
class CenterInformedVoter(BaseVoter):
    '''
    CenterInformedVoter tries to center the choice on what the voter believes will be the outcome, 
    as long as there is no negative payoff
    '''
    
    def __init__(self, pol_x: float, pol_y: float, name):
        
        '''
        Initialize CenterInformedVoter
        '''
        
        BaseVoter.__init__(self, pol_x, pol_y, name)
        self.voter_type = 'CenterInformedVoter'

    def _process_candidates(self, candidatelist):
        """ process candidates, save to self.candidate_ranking """
        
        self.candidate_ranking = {candidate : self.calculate_distance(candidate) for candidate in candidatelist}
        
        return
    

    def vote_choice(self, candidatelist, voterchoice):
        """
        param voterchoice: the result of process_all_voters, candidate list with number of closest voters
                
        return: candidate choice 
        """
        self._process_candidates(candidatelist)
        
        c = Counter(self.candidate_ranking)
        # finding 3 closest values
        chigh = c.most_common()[::-1]
        if len(chigh) > 3:
            chigh3 = chigh[:3]
        else:
            chigh3 = chigh
        
        v = Counter(voterchoice)
        vhigh = v.most_common()

        self.candidate_choice = None # reset
        
#         print(vhigh)
        
        for i in vhigh:
#             print(i[0])
            if self.calculate_reward(i[0]) > 0:
                self.candidate_choice = i[0]
                return i[0]

In [6]:
class PrimaryOptimizingVoter(BaseVoter):
    '''
    PrimaryOptimizingVoter tries to center the choice on what the voter believes will be the outcome
    '''
    
    def __init__(self, pol_x: float, pol_y: float, name):
        
        '''
        Initialize PrimaryOptimizingVoter
        '''
        
        BaseVoter.__init__(self, pol_x, pol_y, name)
        self.voter_type = 'PrimaryOptimizingVoter'

    def _process_candidates(self, candidatelist):
        """ process candidates, save to self.candidate_ranking """
        
        self.candidate_ranking = {candidate : self.calculate_distance(candidate) for candidate in candidatelist}
        
        return
    

    def vote_choice(self, candidatelist, voterchoice):
        """
        param voterchoice: the result of process_all_voters, candidate list with number of closest voters
        
        if the voter's candidate is in the top 3, select that candidate. Else select the next closest candidate in the top
        
        return: candidate choice 
        """
        self._process_candidates(candidatelist)
        
        c = Counter(self.candidate_ranking)
        # finding 3 lowest values
        chigh = c.most_common()[::-1]
        if len(chigh) > 3:
            chigh3 = chigh[:3]
        else:
            chigh3 = chigh
        
        v = Counter(voterchoice)
        vhigh = v.most_common(3) # finding 3 highest values

        self.candidate_choice = None # reset
        
#         print(chigh3)
#         print(vhigh)
        
        for i in vhigh:
            for j in chigh3:
                if i[0] == j[0]:
                    self.candidate_choice = i[0]
#                     print(self.candidate_choice)
                    return i[0]
        
        # not in top 3
        
        for i in vhigh:
            for j in chigh:
                if i[0] == j[0]:
                    self.candidate_choice = i[0]
#                     print(self.candidate_choice)
                    return i[0]


In [7]:
# unit tests

a = Candidate(3, 2, "A", 0)
b = Candidate(5, 4, "B", 0)
c = Candidate(35, 34, "C", 0)
d = Candidate(15, 14, "D", 0)
e = Candidate(1, 1, "E", 0)


j = PrimaryOptimizingVoter(5.5,2.5, "J")
k = PrimaryOptimizingVoter(1.5,2.5, "K")
l = PrimaryOptimizingVoter(2.5, .5, "L")
m = PrimaryOptimizingVoter(3.5, .5, "M")
n = PrimaryOptimizingVoter( .5, .5, "N")
o = PrimaryOptimizingVoter(5.5, 5.5, "O")
p = GreedyVoter(2, 1, "P")
q = GreedyVoter(0, 0, "Q")
r = GreedyVoter(3, 4, "R")
s = CenterInformedVoter(5.5, 4, "S")
t = CenterInformedVoter(1.5, 4, "T")
u = CenterInformedVoter(0, 4, "U")
v = CenterInformedVoter(0, 0, "V")

def _calculate_distance(x: float, y: float, pol_x: float, pol_y: float):
    """ calculate the distance between two (x,y) points """
    distance = ( (x - pol_x)**2 + (y - pol_y)**2 )**(0.5)
    return distance

def process_all_voters(voterlist, candidatelist):
    """
    :param voterlist: list of all voter instances
    :param candidatelist: list of all candidate instances
    
    :return: candidate list with number of closest voters
    """
    ls = [0] * len(candidatelist)
    for v in voterlist:
        choice = None
        scores = [_calculate_distance(v.pol_x, v.pol_y, c.pol_x, c.pol_y) for c in candidatelist]
        index_min = np.argmin(scores)
        ls[index_min] = ls[index_min] + 1
        
    return dict(zip(candidatelist, ls))

voterchoice = process_all_voters([j,k,l,m,n,o,p,q,r,s,t,u,v], [a, b, c, d, e])

j.vote_choice([a, b, c, d, e], voterchoice)
p.vote_choice([a, b, c, d, e])
s.vote_choice([a, b, c, d, e], voterchoice)

for voter in [j,k,l,m,n,o,p,q,r,s,t,u,v]:
    voter.vote_choice([a, b, c, d, e], voterchoice)
    print(voter, voter.candidate_choice)

Voter(PrimaryOptimizingVoter, J, 5.5, 2.5, 0) Candidate(A, 3, 2, 0)
Voter(PrimaryOptimizingVoter, K, 1.5, 2.5, 0) Candidate(A, 3, 2, 0)
Voter(PrimaryOptimizingVoter, L, 2.5, 0.5, 0) Candidate(A, 3, 2, 0)
Voter(PrimaryOptimizingVoter, M, 3.5, 0.5, 0) Candidate(A, 3, 2, 0)
Voter(PrimaryOptimizingVoter, N, 0.5, 0.5, 0) Candidate(A, 3, 2, 0)
Voter(PrimaryOptimizingVoter, O, 5.5, 5.5, 0) Candidate(A, 3, 2, 0)
Voter(GreedyVoter, P, 2, 1, 0) Candidate(C, 35, 34, 0)
Voter(GreedyVoter, Q, 0, 0, 0) Candidate(C, 35, 34, 0)
Voter(GreedyVoter, R, 3, 4, 0) Candidate(C, 35, 34, 0)
Voter(CenterInformedVoter, S, 5.5, 4, 0) Candidate(B, 5, 4, 0)
Voter(CenterInformedVoter, T, 1.5, 4, 0) None
Voter(CenterInformedVoter, U, 0, 4, 0) None
Voter(CenterInformedVoter, V, 0, 0, 0) None


In [8]:
voterchoice = {"A": 3, "B": 5, "C": 2}
remaining_candidates = voterchoice
lowest = min(remaining_candidates, key=remaining_candidates.get)
remaining_candidates.pop(lowest)

2

In [9]:
class MischiefVoter(PrimaryOptimizingVoter):
    '''
    The MischiefVoter is a Primary Optimizing Voter who, however, optimizes if the vote can 
    push an extreme option to the top who will not win the general election
    
    The MischiefVoter runs an analysis of the top 3 options and runs a 
    short process to analyze if an extreme option (out of 3) can 
    be pushed to the top 2, the other has a better chance of winning
    '''
    
    def __init__(self, pol_x: float, pol_y: float, name):
        
        '''
        Initialize MischiefVoter
        '''
        
        PrimaryOptimizingVoter.__init__(self, pol_x, pol_y, name)
        self.voter_type = 'MischiefVoter'

        
#     def _recalibrate(self, voterchoice, candidates):
#         # original voter choice migrates to next closest candidate
#         # elimination rounds
        
#         remaining_candidates = voterchoice
        
#         while len(remaining_candidates) > 3:
#             # get index of lowest candidate
#             lowest = min(remaining_candidates, key=remaining_candidates.get)
#             transfer = remaining_candidates.pop(lowest)
        


    def vote_choice(self, candidatelist, voterchoice):
        """
        param voterchoice: the result of process_all_voters, candidate list with number of closest voters
        
        if the voter's candidate is in the top 3, select that candidate. Else select the next closest candidate in the top
        
        return: candidate choice 
        """
        self._process_candidates(candidatelist)
        
        c = Counter(self.candidate_ranking)
        # finding 3 lowest values
        chigh = c.most_common()[::-1]
        if len(chigh) > 3:
            chigh3 = chigh[:3]
        else:
            chigh3 = chigh
        
        v = Counter(voterchoice)
        vhigh = v.most_common(3) # finding 3 highest values

        self.candidate_choice = None # reset
        
        for i in vhigh:
            for j in chigh3:
                if i[0] == j[0]:
                    self.candidate_choice = i[0]
#                     print(self.candidate_choice)
                    return i[0]
                
        for i in vhigh:
            for j in chigh:
                if i[0] == j[0]:
                    self.candidate_choice = i[0]
#                     print(self.candidate_choice)
                    return i[0]


In [10]:
# if there is a tie, then there is no score, move on to next round - track ties

class Runner(object):
    def __init__(self,
                 n_greedy=300, n_optimizing=500, n_center=300, dist_type_v="uniform", dist_type_c="uniform",
                 reelection=.75, c_min = 2, c_max = 10,
                 alpha=0.0375, mu=0, mu_2 = 0, delta=0.025, sigma=0.1, c_lambda=1, run_steps=100, seed=500):
        
        self.n_voters = n_greedy + n_optimizing + n_center
        self.n_greedy = n_greedy
        self.n_optimizing = n_optimizing
        self.n_center = n_center
        self.dist_type_v = dist_type_v
        self.dist_type_c = dist_type_c
        self.reelection = reelection # percent chance the incumbent chooses to run for reelection
        self.alpha = alpha
        self.mu = mu # mean
        self.mu_2 = mu_2
        self.delta = delta
        self.sigma = sigma # standard deviation
        self.c_lambda = c_lambda
        self.run_steps = run_steps
        
        self.c_min = c_min
        self.c_max = c_max
        
        self.voters = self.make_voters(self.n_greedy, self.n_optimizing, self.n_center, self.dist_type_v)
        self.candidates = None
        
        np.random.seed(seed) # always stable first voters, candidates order
        
    def _make_distribution(self, n_voters, dist_type):
        """ make the specified distribution of voters """
        if dist_type == "uniform":
            voter_dist = np.random.uniform(low=-1, high=1, size=(n_voters, 2))
            return voter_dist
        
        if dist_type == "normal":
            voter_dist = np.random.normal(self.mu, self.sigma, (n_voters, 2))
            return voter_dist
        
        if dist_type =="bimodal_normal":
            voter_dist_a = np.random.normal(self.mu, self.sigma, (math.floor(n_voters/2), 2))
            voter_dist_b = np.random.normal(self.mu, self.sigma, (math.ceil(n_voters/2), 2))
            return np.concatenate((voter_dist_a, voter_dist_b), axis=0)
        
    def generate_candidates(self, dist_type):
        pass
    
    def make_center_voter_array(self, n_center, dist_type):
        
        if n_center == 0:
            return np.array([])
        voter_dist = self._make_distribution(n_center, dist_type)
        voters_list = ['cv%i' % i for i in range(n_center)]
        voters = np.array([CenterInformedVoter(x, y, i) for (x,y),i in zip(voter_dist,voters_list)])
        
        return voters
    
    def make_greedy_voter_array(self, n_greedy, dist_type):
        
        if n_greedy == 0:
            return np.array([])
        
        voter_dist = self._make_distribution(n_greedy, dist_type)
        voters_list = ['gv%i' % i for i in range(n_greedy)]
        voters = np.array([GreedyVoter(x, y, i) for (x,y),i in zip(voter_dist,voters_list)])
        
        return voters
                            
    def make_optimizing_voter_array(self, n_optimizing, dist_type):

        if n_optimizing == 0:
            return np.array([])
        
        voter_dist = self._make_distribution(n_optimizing, dist_type)
#         print(voter_dist)
        voters_list = ['ov%i' % i for i in range(n_optimizing)]
#         print(voters_list)
        voters = np.array([PrimaryOptimizingVoter(x, y, i) for (x,y),i in zip(voter_dist,voters_list)])
        
        return voters
    
   
    def make_voters(self, n_greedy, n_optimizing, n_center, dist_type):
                
        optimizers = self.make_optimizing_voter_array(n_optimizing, dist_type)
        greedy = self.make_greedy_voter_array(n_greedy, dist_type)
        center = self.make_center_voter_array(n_center, dist_type)
        
        all_voters = np.hstack((optimizers, greedy, center))

        return all_voters
        
    def get_voters(self):
        return self.voters
    
    def get_candidate_count(self):
        if type(self.candidates) == None:
            return 0

        return len(self.candidates)
    

    def process_all_voters(self, voters, candidates):
        """
        :param voters: list of all voter instances
        :param candidates: list of all candidate instances

        :return: candidate list with number of closest voters
        """
        ls = [0] * len(candidates)
        for v in voters:
            choice = None
            scores = [_calculate_distance(v.pol_x, v.pol_y, c.pol_x, c.pol_y) for c in candidates]
            index_min = np.argmin(scores)
            ls[index_min] = ls[index_min] + 1

        return dict(zip(candidates, ls))


    def _make_new_candidates(self, c_min, c_max, dist_type, reelection):
        
        n_candidates = np.random.randint(2, 11)
#         n_candidates = 4
        
        reelection = np.random.choice((0,1), n_candidates, p=(1-reelection, reelection))
        
        candidate_dist = self._make_distribution(n_candidates, dist_type)
        candidate_list = ['C%i' % i for i in range(n_candidates)]
        
        candidates = np.array([Candidate(x, y, i, r) for (x,y),i,r in zip(candidate_dist,candidate_list, reelection)])
        self.candidates = candidates
        return candidates
        
    def _run_primary_voting(self):        
        closest_candidates = self.process_all_voters(self.voters, self.candidates)
        for v in self.voters:
            res = v.vote_choice(self.candidates, closest_candidates)

        ls = {c : 0 for c in self.candidates}
        
#         print(ls)
        
        for v in self.voters:
            if v.candidate_choice is not None:
                vchoice = v.candidate_choice
#                 print(vchoice)
                ls[vchoice] += 1
        
        return ls
    
    def _run_general_voting(self, primary_results):
                
        top_2 = Counter(primary_results).most_common(2)
        top_2_ls = [c[0] for c in top_2]
        
#         print(top_2_ls)

        ls = {c : 0 for c in top_2_ls}
        
        for v in self.voters:
            res = v.closest_candidate(top_2_ls)
            ls[res] += 1
        
        return ls
    

    def _run_voting(self):
        self._make_new_candidates(self.c_min, self.c_max, self.dist_type_c, self.reelection)
        
        primary_results = self._run_primary_voting()
    
        results = self._run_general_voting(primary_results)
        
        return Counter(results)
    
    def vote_cycle(self):
        results = self._run_voting()
        winner = max(results, key=results.get)
        
        for v in self.voters:
            v.reward(winner)
    
    def run(self):
        for i in range(self.run_steps):
            self.vote_cycle()
    
    def aggregate_scores(self):
        
        optimizers = [i.current_score() for i in self.voters if i.voter_type == "PrimaryOptimizingVoter"]
        greedy = [i.current_score() for i in self.voters if i.voter_type == "GreedyVoter"]
        center = [i.current_score() for i in self.voters if i.voter_type == "CenterInformedVoter"]

        optimizer_avg = sum(optimizers) / len(optimizers)
        greedy_avg = sum(greedy) / len(greedy)
        center_avg = sum(center) / len(center)
        
        return optimizer_avg, greedy_avg, center_avg


In [11]:
Counter([a,b]).most_common()

[(Candidate(A, 3, 2, 0), 1), (Candidate(B, 5, 4, 0), 1)]

In [12]:
type([a,b])

list

In [876]:
j.voters[0].closest_candidate([a,b])

Candidate(B, 5, 4, 0)

In [13]:
j = Runner()
j.get_voters()
j.run()
# j.candidates
j.get_voters()
j.aggregate_scores()
# j.voters

# j.process_all_voters(j.voters, j.candidates)

# j.vote_result()

(-95.23877878697743, -99.24173392636852, -94.16827195318683)

In [14]:
j.voters[0].pol_x

0.850332236544949

In [789]:
j.voters[15].candidate_choice

Candidate(C3, -0.17362649103280542, -0.030718603772183762, 0)

In [628]:
class IncumbentCenterInformedVoter(CenterInformedVoter):
    def __init__(self, pol_x: float, pol_y: float, name):
        
        '''
        Initialize IncumbentCenterInformedVoter
        '''
        
        PrimaryOptimizingVoter.__init__(self, pol_x, pol_y, name)
        self.voter_type = 'IncumbentCenterInformedVoter'
        
        
    def vote_choice(self, candidatelist, voterchoice):
        """
        param voterchoice: the result of process_all_voters, candidate list with number of closest voters
                
        return: candidate choice 
        """
        self._process_candidates(candidatelist)
        
        c = Counter(self.candidate_ranking)
        # finding 3 closest values
        chigh = c.most_common()[::-1]
        if len(chigh) > 3:
            chigh3 = chigh[:3]
        else:
            chigh3 = chigh
        
        v = Counter(voterchoice)
        vhigh = v.most_common()

        self.candidate_choice = None # reset
        
#         print(vhigh)
        
        for i in vhigh:
#             print(i[0])
            if self.calculate_reward(i[0]) > 0:
                self.candidate_choice = i[0]
                return i[0]        